In [13]:
from tdoa_py.tdoa import *

In [14]:
# Crear el objeto de carga de audio
audio = AudioLoader("audios", "audio_corto.wav")
audio.load_audio()

# Crear el array de micrófonos dentro de la sala
mic_array = MicrophoneArray(n_mics=4, mic_origin=[5, 5, 1.5], displacement=0.5, axis='x')

# Configurar la sala e inmersión del array de micrófonos
room = RoomSimulation(room_dim=[10, 7, 3], rt60=0.4, microphone_array=mic_array, audio_loader=audio)
room.setup_room()
room.add_source(source_position=[4, 2, 1.2])
room.simulate()

# Obtener señales sin guardar archivos
signals = room.room.mic_array.signals  

# Estimar dirección de arribo en memoria
avg_angle, avg_tdoa = DirectionEstimator.estimate_doa(signals, mic_d=0.5, fs=audio.fs)
print(f"Ángulo estimado: {avg_angle:.2f}°, TDOA estimado: {avg_tdoa:.6f} s")

# audio.visualize_audio()



Audio cargado: audios\audio_corto.wav, Frecuencia de muestreo detectada: 48000 Hz
Ángulo estimado: 192.28°, TDOA estimado: -0.012361 s
